# Quantum Brainathon 2025 - EEG Attention Classification

This notebook implements a quantum computing approach to classify different types of attention based on EEG data.

## Table of Contents
1. Setup and Imports
2. Data Loading and Preprocessing
3. Classical Feature Extraction
4. Quantum Feature Mapping
5. Quantum Classification
6. Results Analysis and Visualization
7. Performance Comparison

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Quantum imports
from qiskit import QuantumCircuit, Aer, execute
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms import QSVC

# Custom modules
from data_loader import EEGDataLoader
from preprocessing import preprocess_eeg
from feature_extraction import extract_band_powers
from quantum_ops import QuantumFeatureMapper, QuantumKernelClassifier
from quantum_config import get_quantum_config, print_config_summary

# Set random seed
np.random.seed(42)

# Load quantum configuration
quantum_config = get_quantum_config()
print_config_summary()

ModuleNotFoundError: No module named 'numpy'

In [ ]:
# Initialize data loader
loader = EEGDataLoader()

# Load data
print("Loading EEG data...")
all_subjects = loader.load_all_subjects()
X, y = loader.get_all_paradigm_data(all_subjects)

# Preprocess data
print("\nPreprocessing EEG signals...")
X_preprocessed = preprocess_eeg(X)

# Display data summary
print(f"\nDataset Summary:")
print(f"Number of samples: {len(X)}")
print(f"Number of channels: {X.shape[1]}")
print(f"Unique paradigms: {np.unique(y)}")

In [ ]:
# Extract classical features
def extract_features(data):
    features = []
    for channel in data.columns:
        powers = extract_band_powers(data[channel])
        features.extend(list(powers.values()))
    return np.array(features)

print("Extracting classical features...")
X_features = np.array([extract_features(X_preprocessed.iloc[i:i+1]) 
                      for i in range(len(X_preprocessed))])
print(f"Extracted {X_features.shape[1]} features per sample")

In [ ]:
# Initialize quantum feature mapper
n_qubits = quantum_config['quantum_params']['n_qubits']
feature_mapper = QuantumFeatureMapper(
    n_qubits=n_qubits,
    reps=quantum_config['quantum_params']['feature_map_reps']
)

# Map features to quantum states
print("Mapping features to quantum states...")
X_quantum = feature_mapper.batch_encode_features(X_features)

# Visualize quantum circuit
print("\nQuantum Feature Map Circuit:")
feature_mapper.feature_map.draw(output='mpl')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Prepare data
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_quantum, y_encoded, 
    test_size=0.2, 
    random_state=42
)

# Initialize and train quantum classifier
classifier = QuantumKernelClassifier(
    n_qubits=n_qubits,
    optimizer_config=quantum_config['optimizer']
)

print("Training quantum classifier...")
classifier.train(X_train, y_train)

# Make predictions
y_pred = classifier.predict(X_test)

# Calculate accuracy
accuracy = np.mean(y_pred == y_test)
print(f"\nQuantum Classification Accuracy: {accuracy:.4f}")

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Plot confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d',
            xticklabels=le.classes_,
            yticklabels=le.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Train classical model for comparison
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)
rf_pred = rf_classifier.predict(X_test)
rf_accuracy = np.mean(rf_pred == y_test)

# Compare results
print("Performance Comparison:")
print(f"Quantum Classifier Accuracy: {accuracy:.4f}")
print(f"Classical Classifier Accuracy: {rf_accuracy:.4f}")

# Plot comparison
plt.figure(figsize=(8, 6))
plt.bar(['Quantum', 'Classical'], [accuracy, rf_accuracy])
plt.title('Classifier Performance Comparison')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
plt.show()